# Week 6 Lab: Gold Layer

In this lab you will build the Gold layer — a **star schema** optimized for analytics.

You'll transform the normalized Silver tables into:
- **Dimension tables** with surrogate keys (dim_customer, dim_book, dim_store, dim_date)
- A **fact table** (fact_sales) that references dimensions via foreign keys and contains measures for aggregation

This is the layer that analysts and BI tools query directly.

## Gold Data Model — Star Schema

```
                        ┌───────────────────┐
                        │ gold.dim_customer │
                        ├───────────────────┤
                        │ PK customer_id    │
                        │    email          │
                        │    name           │
                        │    address        │
                        │    city, state    │
                        │    zip            │
                        └─────────▲─────────┘
                                  │
┌──────────────────┐    ┌─────────┴─────────┐    ┌──────────────────┐
│  gold.dim_book   │◄───┤  gold.fact_sales  ├───►│  gold.dim_store  │
├──────────────────┤    ├───────────────────┤    ├──────────────────┤
│ PK book_id       │    │ PK sales_id       │    │ PK store_id      │
│    isbn          │    │ FK customer_id    │    │    store_nbr     │
│    title         │    │ FK book_id        │    │    name          │
│    author        │    │ FK date_id        │    │    address       │
│    genre         │    │ FK store_id       │    │    city, state   │
└──────────────────┘    │    order_id       │    │    zip           │
                        │    order_channel  │    └──────────────────┘
                        │    isbn           │
                        │    quantity       │
                        │    unit_price     │
                        │    line_total     │
                        │    payment_method │
                        └─────────┬─────────┘
                                  │
                         ┌────────▼─────────┐
                         │  gold.dim_date   │
                         ├──────────────────┤
                         │ PK date_id       │
                         │    full_date     │
                         │    day_name      │
                         │    month         │
                         │    quarter       │
                         │    year          │
                         │    (+ 15 more)   │
                         └──────────────────┘
```

---
## Prerequisites

Before running this notebook:
1. Run the DDL notebook at `ddl/gold` to create the target Gold tables.
2. Make sure the Silver layer is populated (Week 5 lab).

---
## Step 1: Build gold.dim_customer (SCD Type 1)

Source: `silver.customers`

This dimension uses **SCD Type 1** (Slowly Changing Dimension, Type 1) — when a customer's information changes, we simply overwrite the existing row. No history is kept.

The `customer_id` column is a **surrogate key** generated automatically by `GENERATED ALWAYS AS IDENTITY` on the table. We never set it manually — the database assigns it on INSERT. On UPDATE (when the customer already exists), we update all descriptive fields but the surrogate key stays the same.

We also add a **sentinel row** for `'in-store'` — this represents anonymous in-store customers who didn't provide an email. Without this row, fact table JOINs would drop those orders.

Merge customers from `silver.customers` into `gold.dim_customer`, matching on the natural key `email`. Since the table has an identity column, we must list columns explicitly (no `INSERT *`).

In [ ]:
MERGE INTO gold.dim_customer AS target
USING silver.customers AS source
ON target.email = source.email
WHEN MATCHED THEN UPDATE SET
  target.name = source.name,
  target.address = source.address,
  target.city = source.city,
  target.state = source.state,
  target.zip = source.zip
WHEN NOT MATCHED THEN INSERT (email, name, address, city, state, zip)
  VALUES (source.email, source.name, source.address, source.city, source.state, source.zip)

Insert a sentinel row for anonymous in-store customers. The MERGE ensures this only runs once — if the row already exists, it's a no-op.

In [ ]:
MERGE INTO gold.dim_customer AS target
USING (SELECT 'in-store' AS email, 'In-Store Customer' AS name) AS source
ON target.email = source.email
WHEN NOT MATCHED THEN INSERT (email, name)
  VALUES (source.email, source.name)

---
## Step 2: Build gold.dim_store

Source: `silver.stores`

Similar to dim_customer, we also need a **sentinel row** for `'online'` — this represents orders placed through the website with no physical store.

Note: the surrogate key is `store_id`. The natural key from the source system is `store_nbr`.

Merge stores from `silver.stores` into `gold.dim_store`, matching on the natural key `store_nbr`.

In [ ]:
MERGE INTO gold.dim_store AS target
USING silver.stores AS source
ON target.store_nbr = source.store_nbr
WHEN MATCHED THEN UPDATE SET
  target.name = source.name,
  target.address = source.address,
  target.city = source.city,
  target.state = source.state,
  target.zip = source.zip
WHEN NOT MATCHED THEN INSERT (store_nbr, name, address, city, state, zip)
  VALUES (source.store_nbr, source.name, source.address, source.city, source.state, source.zip)

Insert a sentinel row for online orders (no physical store).

In [ ]:
MERGE INTO gold.dim_store AS target
USING (SELECT 'online' AS store_nbr, 'Online' AS name) AS source
ON target.store_nbr = source.store_nbr
WHEN NOT MATCHED THEN INSERT (store_nbr, name)
  VALUES (source.store_nbr, source.name)

---
## Step 3: Build gold.dim_book

Source: `silver.books`

A straightforward dimension — one row per book, with a surrogate `book_id` auto-generated on insert.

Merge books from `silver.books` into `gold.dim_book`, matching on `isbn`.

In [ ]:
MERGE INTO gold.dim_book AS target
USING silver.books AS source
ON target.isbn = source.isbn
WHEN MATCHED THEN UPDATE SET
  target.title = source.title,
  target.author = source.author,
  target.genre = source.genre
WHEN NOT MATCHED THEN INSERT (isbn, title, author, genre)
  VALUES (source.isbn, source.title, source.author, source.genre)

---
## Step 4: Build gold.dim_date

Source: pre-built CSV at `/FileStore/hwe-data/dim_date/dim_date.csv`

The date dimension is a calendar table — one row per date — pre-populated with dates from 2024 through 2029. It has 21 columns covering day name, month name, fiscal periods, and more. This lets analysts filter and group by any calendar attribute without date math in every query.

Unlike the other dimensions, dim_date is loaded from a pre-built CSV rather than derived from Silver data. We use `INSERT OVERWRITE` to fully replace the table contents on each run.

Load the date dimension from the pre-built CSV file using `INSERT OVERWRITE`.

In [ ]:
INSERT OVERWRITE gold.dim_date
SELECT
  CAST(date_id AS INT),
  CAST(full_date AS DATE),
  CAST(day_of_week AS TINYINT),
  CAST(day_num_in_month AS TINYINT),
  day_name,
  day_abbrev,
  weekday_flag,
  CAST(week_num_in_year AS TINYINT),
  CAST(week_begin_date AS DATE),
  CAST(week_begin_date_key AS INT),
  CAST(month AS TINYINT),
  month_name,
  month_abbrev,
  CAST(quarter AS TINYINT),
  CAST(year AS SMALLINT),
  CAST(yearmo AS INT),
  CAST(fiscal_month AS TINYINT),
  CAST(fiscal_quarter AS TINYINT),
  CAST(fiscal_year AS SMALLINT),
  last_day_in_month_flag,
  CAST(same_day_year_ago_date AS DATE)
FROM read_files(
  '/FileStore/hwe-data/dim_date/dim_date.csv',
  format => 'csv',
  header => true
)

---
## Step 5: Build gold.fact_sales

Sources: `silver.orders` + `silver.order_items` + all four dimension tables

The fact table is the center of the star schema. Each row represents one **line item** (one book in one order). It contains:
- **Foreign keys** to each dimension (looked up by joining on natural keys)
- **Measures** for aggregation: `quantity`, `unit_price`, `line_total`
- **Degenerate dimensions**: `order_id`, `order_channel`, `isbn`, and `payment_method` (too simple for their own dimension table)

We build it by:
1. Joining `silver.order_items` to `silver.orders` to get order-level fields
2. Joining to each dimension table to look up the surrogate key
3. Computing `line_total = quantity * unit_price`

The degenerate dimensions `(order_id, order_channel, isbn)` form a **natural key** on the fact table, which lets us use **MERGE** — just like every other table in the pipeline. On re-runs, existing rows are updated and only new order items are inserted.

Merge fact rows from silver into `gold.fact_sales`, matching on the natural key `(order_id, order_channel, isbn)`.

In [ ]:
MERGE INTO gold.fact_sales AS target
USING (
  SELECT
    dc.customer_id,
    db.book_id,
    dd.date_id,
    ds.store_id,
    o.order_id,
    o.order_channel,
    oi.isbn,
    oi.quantity,
    oi.unit_price,
    oi.quantity * oi.unit_price AS line_total,
    o.payment_method
  FROM silver.order_items oi
  JOIN silver.orders o
    ON oi.order_id = o.order_id
    AND oi.order_channel = o.order_channel
  JOIN gold.dim_customer dc
    ON o.customer_email = dc.email
  JOIN gold.dim_book db
    ON oi.isbn = db.isbn
  JOIN gold.dim_date dd
    ON CAST(o.order_datetime AS DATE) = dd.full_date
  JOIN gold.dim_store ds
    ON o.store_nbr = ds.store_nbr
) AS source
ON  target.order_id = source.order_id
AND target.order_channel = source.order_channel
AND target.isbn = source.isbn
WHEN MATCHED THEN UPDATE SET
  target.customer_id = source.customer_id,
  target.book_id = source.book_id,
  target.date_id = source.date_id,
  target.store_id = source.store_id,
  target.quantity = source.quantity,
  target.unit_price = source.unit_price,
  target.line_total = source.line_total,
  target.payment_method = source.payment_method
WHEN NOT MATCHED THEN INSERT (
  customer_id, book_id, date_id, store_id,
  order_id, order_channel, isbn,
  quantity, unit_price, line_total, payment_method
)
VALUES (
  source.customer_id, source.book_id, source.date_id, source.store_id,
  source.order_id, source.order_channel, source.isbn,
  source.quantity, source.unit_price, source.line_total, source.payment_method
)

---
## Step 6: Verify

Let's validate the star schema is correct and working.

**Row count** — `gold.fact_sales` should have the same number of rows as `silver.order_items`.

In [ ]:
SELECT
  (SELECT COUNT(*) FROM gold.fact_sales) AS fact_count,
  (SELECT COUNT(*) FROM silver.order_items) AS silver_order_items_count

**No null foreign keys** — Every row in the fact table should have valid references to all four dimensions. This query should return all zeros.

In [ ]:
SELECT
  SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS null_customer_ids,
  SUM(CASE WHEN book_id IS NULL THEN 1 ELSE 0 END) AS null_book_ids,
  SUM(CASE WHEN date_id IS NULL THEN 1 ELSE 0 END) AS null_date_ids,
  SUM(CASE WHEN store_id IS NULL THEN 1 ELSE 0 END) AS null_store_ids
FROM gold.fact_sales

**line_total check** — Verify that `line_total` equals `quantity * unit_price` for every row. This query should return **no rows**.

In [ ]:
SELECT sales_id, quantity, unit_price, line_total
FROM gold.fact_sales
WHERE line_total != quantity * unit_price

---
## Step 7: Sample Analytics Queries

The whole point of a star schema is to make analytics queries simple and fast. Here are a few examples that demonstrate the pattern: pick a measure from the fact table, join to the dimensions you want to filter/group by.

**Total sales by genre** — Which book genres generate the most revenue?

In [ ]:
SELECT
  b.genre,
  SUM(f.line_total) AS total_sales,
  SUM(f.quantity) AS total_units
FROM gold.fact_sales f
JOIN gold.dim_book b ON f.book_id = b.book_id
GROUP BY b.genre
ORDER BY total_sales DESC

**Monthly sales by store** — How does each store perform over time?

In [ ]:
SELECT
  s.name AS store_name,
  d.year,
  d.month,
  SUM(f.line_total) AS total_sales
FROM gold.fact_sales f
JOIN gold.dim_store s ON f.store_id = s.store_id
JOIN gold.dim_date d ON f.date_id = d.date_id
GROUP BY s.name, d.year, d.month
ORDER BY s.name, d.year, d.month

**Top 10 customers by spending** — Who are our best customers?

In [ ]:
SELECT
  c.name,
  c.email,
  SUM(f.line_total) AS total_spent,
  SUM(f.quantity) AS total_items,
  COUNT(DISTINCT f.date_id) AS order_days
FROM gold.fact_sales f
JOIN gold.dim_customer c ON f.customer_id = c.customer_id
WHERE c.email != 'in-store'
GROUP BY c.name, c.email
ORDER BY total_spent DESC
LIMIT 10

**Sales by channel and quarter** — How do online vs. in-store sales compare across quarters?

In [ ]:
SELECT
  f.order_channel,
  d.year,
  d.quarter,
  SUM(f.line_total) AS total_sales,
  COUNT(*) AS line_items
FROM gold.fact_sales f
JOIN gold.dim_date d ON f.date_id = d.date_id
GROUP BY f.order_channel, d.year, d.quarter
ORDER BY d.year, d.quarter, f.order_channel